In [1]:
inputPath1 = "/data/students/bigdata_internet/lab3/register.csv"

In [2]:
inputPath2 = "/data/students/bigdata_internet/lab3/stations.csv"

In [3]:
df_register = spark.read.load(inputPath1,sep ="\t",format = "csv",header =True , inferscherma=True)

In [4]:
#number of total before editing

In [5]:
df_register.count()

25319028

In [6]:
df_register.createOrReplaceTempView("st")

In [7]:
filtered_reg = spark.sql("SELECT * FROM st WHERE NOT (used_slots = 0 AND free_slots = 0)")

In [8]:
filtered_reg.count() # number of rows after data cleaning

25104121

In [9]:
df_station = spark.read.load(inputPath2,sep = "\t" ,format = "csv",header = True)

In [10]:
withTimeSlot = filtered_reg.selectExpr("station","(date_format(timestamp,'EEEE'),hour(timestamp)) AS Tj ",
                     "used_slots","free_slots")

In [11]:
withTimeSlot.createOrReplaceTempView("sttm")

In [12]:
allReadings = spark.sql("SELECT station,Tj,count(station) AS countAll FROM sttm GROUP BY station,Tj")

In [13]:
allReadings.createOrReplaceTempView("AllReadings")

In [14]:
criticalsReadings = spark.sql(" SELECT station,Tj,COUNT(station) AS countCriticals FROM sttm WHERE free_slots = 0 GROUP BY station, Tj")

In [15]:
criticalsReadings.createOrReplaceTempView("CriticalReadings")

In [16]:
joinedTable = spark.sql("""
    SELECT c.station, c.Tj, c.countCriticals, A.countAll
    FROM CriticalReadings c ,AllReadings A
    WHERE c.station = A.station
    AND c.Tj = A.Tj
""")

In [17]:
joinedTable.createOrReplaceTempView("joinedTable")

In [18]:
SiTj = spark.sql("""
    SELECT station, Tj, countCriticals / countAll AS Csitj 
    FROM joinedtable 
""")

In [20]:
SiTj.show(5) #2.1 for each pair(station,Tj) the criticality value is calculated

[Stage 9:====================================================>    (11 + 1) / 12]

+-------+--------------+--------------------+
|station|            Tj|               Csitj|
+-------+--------------+--------------------+
|     10|[Wednesday, 2]| 0.15463917525773196|
|    100|[Saturday, 14]| 0.01763668430335097|
|    102|  [Tuesday, 7]|  0.0106951871657754|
|    109|   [Sunday, 2]|0.013888888888888888|
|     11|  [Monday, 10]| 0.20934579439252338|
+-------+--------------+--------------------+
only showing top 5 rows



In [21]:
aboveThreshold = SiTj.filter('Csitj > 0.6') ##2.2 only critical pairs with above minimum criticality threshold(0.6) are selected

In [22]:
aboveThreshold.createOrReplaceTempView("above")

In [23]:
# I change the day of weeks into numbers

In [69]:
daysWithNumber = spark.sql('''SELECT 
    station,
    CASE 
        WHEN Tj.col1 = "Friday" THEN struct(5 AS col1, Tj.col2)
        WHEN Tj.col1 = "Monday" THEN struct(1 AS col1, Tj.col2)
        WHEN Tj.col1 = "Saturday" THEN struct(6 AS col1, Tj.col2)
    END AS Tj,
    Csitj
FROM 
    above''')
daysWithNumber.createOrReplaceTempView("ordered")

In [25]:
#2.3 now orderin them by criticality value,station id,day of week and hour

In [70]:
ordered = spark.sql('SELECT * FROM ordered ORDER BY Csitj,station,Tj.col1 ,Tj.col2')

In [77]:
ordered.createOrReplaceTempView("orderedday")

In [78]:
from pyspark.sql.functions import col,when

daysWithWord = spark.sql('''SELECT 
    station,
    CASE 
        WHEN Tj.col1 = 5 THEN struct("Friday" AS col1, Tj.col2)
        WHEN Tj.col1 = 1 THEN struct("Monday" AS col1, Tj.col2)
        WHEN Tj.col1 = 6 THEN struct("Saturday" AS col1, Tj.col2)
    END AS Tj,
    Csitj
FROM 
    orderedday''')
# now days'number to day of week again

In [80]:
df_station.createOrReplaceTempView("Stations_info")

In [81]:
daysWithWord.createOrReplaceTempView("ordering")

In [82]:
address = spark.sql("""
    SELECT S.id, S.longitude, S.latitude, S.name, O.Tj ,O.Csitj
    FROM Stations_info S
    JOIN ordering O ON S.id = O.station
""")

In [83]:
w = address.selectExpr('id AS `station id`',
                       'name AS `station name`',
                       'longitude AS `station longitude`',
                       'latitude AS `station latitude`',
                       'Tj.col1 AS `day of week`',
                       'Tj.col2 AS `hour`',
                       'Csitj AS `critality value`')

In [86]:
w.show(truncate=False)

+----------+------------------------------+-----------------+----------------+-----------+----+------------------+
|station id|station name                  |station longitude|station latitude|day of week|hour|critality value   |
+----------+------------------------------+-----------------+----------------+-----------+----+------------------+
|9         |MarquÃ¨s de l\'Argentera      |2.185294         |41.385006       |Friday     |10  |0.6129032258064516|
|10        |Avinguda del Marques Argentera|2.185206         |41.384875       |Saturday   |0   |0.622107969151671 |
|58        |Sant Oleguer                  |2.170736         |41.377536       |Monday     |1   |0.6239554317548747|
|9         |MarquÃ¨s de l\'Argentera      |2.185294         |41.385006       |Friday     |22  |0.6258389261744967|
|58        |Sant Oleguer                  |2.170736         |41.377536       |Monday     |0   |0.6323119777158774|
+----------+------------------------------+-----------------+----------------+--

In [87]:
outputFile = "/user/s310279/lab3/outPutBySQL"

In [88]:
#2.4 storing them with header and columns are separated by tab

In [90]:
w.write.option("delimiter", "\t").csv(outputFile, header=True)